# 分散表現の確認
word2vecによる分散表現について学びます。

## コーパスの前処理
前のセクションと同様に、コーパスに前処理を行います。

In [1]:
import re
import pickle
from janome.tokenizer import Tokenizer

with open("wagahaiwa_nekodearu.txt", mode="r", encoding="utf-8") as f:  # ファイルの読み込み
    wagahai_original = f.read()

wagahai = re.sub("《[^》]+》", "", wagahai_original) # ルビの削除
wagahai = re.sub("［[^］]+］", "", wagahai) # 読みの注意の削除
wagahai = re.sub("[｜ 　「」\n]", "", wagahai) # | と全角半角スペース、「」と改行の削除

seperator = "。"  # 。をセパレータに指定
wagahai_list = wagahai.split(seperator)  # セパレーターを使って文章をリストに分割する
wagahai_list.pop() # 最後の要素は空の文字列になるので、削除
wagahai_list = [x+seperator for x in wagahai_list]  # 文章の最後に。を追加
        
t = Tokenizer()

wagahai_words = []
for sentence in wagahai_list:
    wagahai_words.append(t.tokenize(sentence, wakati=True))   # 文章ごとに単語に分割し、リストに格納
    
with open('wagahai_words.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(wagahai_words, f)

保存できていることを確認します。

In [2]:
with open('wagahai_words.pickle', mode='rb') as f:
    wagahai_words = pickle.load(f)

print(wagahai_words[:10])

[['吾輩', 'は', '猫', 'で', 'ある', '。'], ['名前', 'は', 'まだ', '無い', '。'], ['どこ', 'で', '生れ', 'た', 'か', 'とんと', '見当', 'が', 'つか', 'ぬ', '。'], ['何', 'でも', '薄暗い', 'じめじめ', 'し', 'た', '所', 'で', 'ニャーニャー', '泣い', 'て', 'いた事', 'だけ', 'は', '記憶', 'し', 'て', 'いる', '。'], ['吾輩', 'は', 'ここ', 'で', '始め', 'て', '人間', 'という', 'もの', 'を', '見', 'た', '。'], ['しかも', 'あと', 'で', '聞く', 'と', 'それ', 'は', '書生', 'という', '人間', '中', 'で', '一番', '獰悪', 'な', '種族', 'で', 'あっ', 'た', 'そう', 'だ', '。'], ['この', '書生', 'という', 'の', 'は', '時々', '我々', 'を', '捕え', 'て', '煮', 'て', '食う', 'という', '話', 'で', 'ある', '。'], ['しかし', 'その', '当時', 'は', '何', 'という', '考', 'も', 'なかっ', 'た', 'から', '別段', '恐し', 'いとも', '思わ', 'なかっ', 'た', '。'], ['ただ', '彼', 'の', '掌', 'に', '載せ', 'られ', 'て', 'スー', 'と', '持ち上げ', 'られ', 'た', '時', '何だか', 'フワフワ', 'し', 'た', '感じ', 'が', 'あっ', 'た', 'ばかり', 'で', 'ある', '。'], ['掌', 'の', '上', 'で', '少し', '落ちつい', 'て', '書生', 'の', '顔', 'を', '見', 'た', 'の', 'が', 'いわゆる', '人間', 'という', 'もの', 'の', '見', '始', 'で', 'あろ', 'う', '。']]


## word2vecを用いた学習
今回はword2vecのためにライブラリgensimを使います。  
gensimは、様々なトピックモデルを実装したPythonライブラリです。  
トピックモデルとは、潜在的なトピックから文章が確率的に生成されると仮定したモデルです。

gensimについて、詳細は以下のリンクを参考にどうぞ。  
https://radimrehurek.com/gensim/

以下では、word2vecを用いてコーパスの学習を行い、学習済みのモデルを作成します。

In [3]:
from gensim.models import word2vec

# size : 中間層のニューロン数
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# iter : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model = word2vec.Word2Vec(wagahai_words,
                          size=100,
                          min_count=5,
                          window=5,
                          iter=20,
                          sg = 0)

/Users/ichi/anaconda3/envs/nlp_bot/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


分散表現を見ていきましょう。  
重みを表す行列（分散表現）の形状と、行列そのものを表示します。

In [4]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  # 分散表現

(3309, 100)
[[ 0.47850484 -0.50791544  0.40411606 ...  0.83937687  0.2790246
  -0.43256155]
 [-0.18664628  0.5062127  -0.43288234 ... -0.50427926 -0.3086158
   0.1830239 ]
 [ 0.04193473 -1.7947731   0.25922394 ... -0.10832618 -0.31415
  -0.3051706 ]
 ...
 [ 0.04093803 -0.04472658  0.5961911  ... -0.09814879  0.12705179
  -0.14396422]
 [ 0.1553409  -0.0185522   0.11996546 ...  0.06630289 -0.03235835
   0.0818951 ]
 [ 0.06002593 -0.09871671  0.32169864 ...  0.02848747  0.05845167
   0.13607389]]


語彙の数、および語彙の最初の10語を表示します。

In [5]:
print(len(model.wv.index2word))  # 語彙の数
print(model.wv.index2word[:10])  # 最初の10単語

3309
['の', '。', 'て', '、', 'は', 'に', 'を', 'と', 'が', 'た']


語彙における、最初の単語の単語ベクトルを2通りの方法で表示します。

In [6]:
print(model.wv.vectors[0])  # 最初のベクトル
print(model.wv.__getitem__("の"))  # 最初の単語「の」のベクトル

[ 0.47850484 -0.50791544  0.40411606  0.03293316 -0.86857045 -0.24617039
  0.23947564 -0.7427519   0.20454103  0.6878894  -0.8256731  -0.32783347
  0.43075436  0.4852031  -0.5521262  -0.2017192  -0.40648684  0.33448133
 -0.04680029  0.80531716  0.41257033  0.16146965 -0.18052511  0.643233
  0.16804035 -0.33015192 -0.02217991 -0.36687666 -0.13573416  0.384273
  0.60808617  0.00272436  0.5688666   0.38034114  0.8764062  -0.28309602
 -0.15245335 -0.3958195   0.5564369  -0.463331   -0.32916853  0.21449259
 -0.27537405 -0.28018066  0.6446024   0.01511167  0.0149075   0.7395821
  0.60606426  0.55703795 -0.5035908   1.1145527   0.17544214  0.9384086
 -0.6485448  -0.5668605   0.15616375 -0.01977319 -0.383565    0.05587886
 -0.36894622  0.76181865 -0.5607361  -0.2166123  -0.03836256 -0.05110534
  0.13102327  0.3346482  -0.50276893 -0.8666883  -0.8301758  -0.84077436
  1.0365133   0.14950885 -0.46775836 -0.4697836   0.4049342   0.22359566
  0.69291955  0.91060364  1.5678687   0.709839   -0.28769

両者ともに同じベクトルですね。

## 課題:
単語「猫」を単語ベクトルで表してみましょう。  
また、skip-gramも試して結果を比較してみましょう。

In [7]:
print(model.wv.__getitem__('猫'))

[ 0.29037258 -0.5826171   0.44132808 -0.5563748  -0.61536944  0.0137474
 -0.61916286 -0.8961787  -0.22377321  0.2010818  -0.9689285  -0.2610208
  0.29267126  0.43053034  1.1502137  -0.38897416  1.3126229   0.1039951
 -0.65285397 -0.18884562  0.14885017  0.15006222  0.2707275   0.8883237
  0.8885451  -0.24213119  1.5541054  -0.5748787  -0.39538336  0.6785998
  0.61892134 -0.8792448   0.4386257  -0.6232554   0.44513384 -0.28097478
  0.49781808 -0.03949196  1.7105714  -0.2838148  -0.63684845  0.14811006
 -0.6817013   0.96673614  0.6222911  -0.32212147 -1.0005525   0.36366996
 -0.41572848 -0.8203272   0.9231599   0.84458417 -0.674919    1.3766514
 -0.13917176  0.42037132  0.36644366 -0.53313315 -1.0270896   0.37078828
 -0.9624425  -0.48971823 -0.46030697 -0.27020034  0.05121028 -0.38288754
  0.09140068 -0.61657184 -0.92689574 -0.05417521  1.444124    0.6422113
  0.32652622  0.0897107   0.4757801  -0.5600293   0.5599755  -0.014549
  0.06061541 -0.60028106  0.45786914  0.09119079  0.2134896 

In [8]:
# size : 中間層のニューロン数
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# iter : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model_sg = word2vec.Word2Vec(wagahai_words,
                          size=100,
                          min_count=5,
                          window=5,
                          iter=20,
                          sg = 1)

In [9]:
print(len(model_sg.wv.index2word))  # 語彙の数
print(model_sg.wv.index2word[:10])  # 最初の10単語

3309
['の', '。', 'て', '、', 'は', 'に', 'を', 'と', 'が', 'た']


In [10]:
print(model_sg.wv.vectors[0])  # 最初のベクトル
print(model_sg.wv.__getitem__("の"))  # 最初の単語「の」のベクトル

[ 2.49483690e-01 -1.29452959e-01  1.06990390e-01 -2.69696787e-02
 -2.61901736e-01  3.02358996e-02  1.05922021e-01 -1.00710735e-01
  4.92527746e-02 -3.41102853e-02  1.47550702e-01 -9.76501629e-02
  1.24776423e-01  1.62699059e-01 -2.84132920e-02 -1.09884255e-01
  2.73212850e-01  1.49206042e-01 -7.14940652e-02  2.52247959e-01
  2.44211107e-01  8.66199136e-02 -2.02591702e-01 -8.31218809e-02
 -2.91033532e-04 -1.88908830e-01  2.58044600e-01  1.10568389e-01
 -1.92563534e-01  1.54310558e-02  2.39133030e-01  1.79356739e-01
 -2.17291847e-01 -2.41740093e-01 -2.45121750e-03 -2.37056427e-03
  1.81900233e-01 -1.73380941e-01 -3.78175750e-02 -3.36456835e-01
 -1.17146589e-01 -6.93231970e-02 -1.12092666e-01 -1.40436247e-01
  3.39170218e-01  2.30594222e-02 -1.72016338e-01  1.63999617e-01
 -6.09291680e-02  3.08859020e-01 -2.86178850e-03  3.74305576e-01
 -1.24558873e-01  5.40373176e-02 -2.80858837e-02  1.74766704e-01
  2.37317737e-02 -1.67809248e-01  4.16610725e-02  7.29565546e-02
 -6.91422448e-02  2.32171

In [11]:
print(model.wv.__getitem__('猫'))
print(model_sg.wv.__getitem__('猫'))

[ 0.29037258 -0.5826171   0.44132808 -0.5563748  -0.61536944  0.0137474
 -0.61916286 -0.8961787  -0.22377321  0.2010818  -0.9689285  -0.2610208
  0.29267126  0.43053034  1.1502137  -0.38897416  1.3126229   0.1039951
 -0.65285397 -0.18884562  0.14885017  0.15006222  0.2707275   0.8883237
  0.8885451  -0.24213119  1.5541054  -0.5748787  -0.39538336  0.6785998
  0.61892134 -0.8792448   0.4386257  -0.6232554   0.44513384 -0.28097478
  0.49781808 -0.03949196  1.7105714  -0.2838148  -0.63684845  0.14811006
 -0.6817013   0.96673614  0.6222911  -0.32212147 -1.0005525   0.36366996
 -0.41572848 -0.8203272   0.9231599   0.84458417 -0.674919    1.3766514
 -0.13917176  0.42037132  0.36644366 -0.53313315 -1.0270896   0.37078828
 -0.9624425  -0.48971823 -0.46030697 -0.27020034  0.05121028 -0.38288754
  0.09140068 -0.61657184 -0.92689574 -0.05417521  1.444124    0.6422113
  0.32652622  0.0897107   0.4757801  -0.5600293   0.5599755  -0.014549
  0.06061541 -0.60028106  0.45786914  0.09119079  0.2134896 

## 何を比較しろと言っているのかな？